In [27]:
import sys
sys.path.append("../../")
import os
import datetime
import pandas as pd
import numpy as np
from example_reactiondiffusion import get_rd_data
from sindy_utils import library_size
from training import train_network
import torch

# Generate data

In [28]:
training_data_, validation_data_, test_data = get_rd_data()

# Function to convert data to tensors and move to GPU
def convert_to_tensors(data_dict, device):
    for key, value in data_dict.items():
        if value is not None:
            data_dict[key] = torch.tensor(value).float().to(device)
    return data_dict

# Specify the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Convert training and validation data to tensors and move to GPU
training_data = convert_to_tensors(training_data_, device)
validation_data = convert_to_tensors(validation_data_, device)

# If 'ddx' is required and should be set to None
training_data['ddx'] = None
validation_data['ddx'] = None

# Set up model and training parameters

In [29]:
params = {}
params['input_dim'] = training_data['y1'].numel()*training_data['y2'].numel()
params['latent_dim'] = 2
params['model_order'] = 1
params['poly_order'] = 3
params['include_sine'] = True
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_z'] = 0.01
params['loss_weight_sindy_x'] = 0.5
params['loss_weight_sindy_regularization'] = 0.1

params['activation'] = 'sigmoid'
params['widths'] = [256]

# training parameters
params['epoch_size'] = training_data['t'].numel()
params['batch_size'] = 1000
params['learning_rate'] = 1e-3

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 3001
params['refinement_epochs'] = 1001

# Run training experiments

In [30]:
num_experiments = 10
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = torch.ones((params['library_dim'], params['latent_dim'])).to("cuda:0")

    params['save_name'] = 'rd_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    #tf.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    # df = df.append({**results_dict, **params}, ignore_index=True)
    df = pd.concat([df, pd.DataFrame([{**results_dict, **params}])], ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
TRAINING


c:\Users\zhaox\OneDrive - Delft University of Technology\桌面\Q4\reproductory\report\repo_sindy-main\torch_result\rd\../..\training.py:224: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'x': torch.tensor(data['x'][idxs], dtype=torch.float32).to(device),
c:\Users\zhaox\OneDrive - Delft University of Technology\桌面\Q4\reproductory\report\repo_sindy-main\torch_result\rd\../..\training.py:225: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'dx': torch.tensor(data['dx'][idxs], dtype=torch.float32).to(device)
c:\Users\zhaox\OneDrive - Delft University of Technology\桌面\Q4\reproductory\report\repo_sindy-main\torch_result\rd\../..\training.py:232: UserWarning: To copy construct from a tensor, it is r

Epoch 0
   Training Losses:
      Total Loss: 0.44815441966056824
      decoder Loss: 0.17480897903442383
      sindy_z Loss: 10.60834789276123
      sindy_x Loss: 0.13522449135780334
      sindy_regularization Loss: 0.9964969158172607
   Validation Losses:
      Total Loss: 0.4494081437587738
      decoder Loss: 0.17471599578857422
      sindy_z Loss: 10.736557006835938
      sindy_x Loss: 0.1353537142276764
      sindy_regularization Loss: 0.9964969158172607
Decoder Loss Ratio: 1.103888, Decoder SINDy Loss Ratio: 1.037172
Epoch 100
   Training Losses:
      Total Loss: 0.16695299744606018
      decoder Loss: 0.05220843106508255
      sindy_z Loss: 0.4230600893497467
      sindy_x Loss: 0.04619520530104637
      sindy_regularization Loss: 0.8741636276245117
   Validation Losses:
      Total Loss: 0.16941338777542114
      decoder Loss: 0.05384118854999542
      sindy_z Loss: 0.424608051776886
      sindy_x Loss: 0.047819528728723526
      sindy_regularization Loss: 0.8741636276245117


KeyboardInterrupt: 